In [1]:
import pandas as pd
import math
import re

In [2]:
re_han = re.compile("([\u4E00-\u9FD5]+)", re.U)

def txtcnt(txt):
    cnt = 0
    blocks = re_han.split(txt)
    for blk in blocks:
        if not blk:
            continue
        if re_han.match(blk):
            cnt += len(blk)
    return cnt

def tokens(txt,word_len):
    words = []
    blocks = re_han.split(txt)
    for blk in blocks:
        if not blk:
            continue
        if re_han.match(blk):
            words += ngram(blk,word_len)
    return words

def ngram(txt,word_len):
    words = []
    tmp_txt = txt
    remain_len = len(tmp_txt) - word_len
    if remain_len <= 0:
        words.append(tmp_txt)
    else:
        inx = 0
        while inx <= remain_len:
            word = tmp_txt[inx:inx+word_len]
            words.append(word)
            inx += 1
    return words

In [3]:
word_len = 2
line_inx = 1
file_name = 'comment_word_'+ str(word_len) +'.txt'
with open('comment_all.txt','r') as f \
    ,open(file_name,'w') as w:
    for line in f.readlines():
        words = tokens(line,word_len)
        for word in words:
            w.write(word + ',' + str(line_inx) +',1\n')
        line_inx += 1

In [4]:
df = pd.read_csv(file_name,header=None)
df[:2]

,0,1,2
0,好好,1,1
1,好吃,1,1


In [5]:
line_inx

4196

In [6]:
idf = df.filter(items=[0,1])

In [7]:
len(idf)

42023

In [8]:
idf_df = idf.drop_duplicates()

In [9]:
idf_df1 = idf_df.groupby(by=0).count()

In [10]:
idf_df1[:5]

,1
0,
一一,2
一下,11
一个,74
一些,3
一人,11


In [11]:
tf = df.filter(items=[0,2])

In [12]:
tf_df = tf.groupby(by=0).sum()

In [13]:
tf_df[:5]

,2
0,
一一,2
一下,12
一个,86
一些,5
一人,11


In [14]:
tf_df.sort_values(by=2,ascending=False).to_csv('词频.csv')

In [15]:
tf_idf = idf_df1.join(tf_df)

In [16]:
idf_max = tf_idf[1].max()+1
tf_max = tf_idf[2].max()+1

tf_idf['idf'] = tf_idf[1].apply(lambda x: math.log(idf_max/x))
tf_idf['tf'] = tf_idf[2].apply(lambda x: x/tf_max)

In [17]:
tf_idf['score'] = tf_idf['idf'] * tf_idf['tf']

In [18]:
tf_idf[:5]

,1,2,idf,tf,score
0,,,,,
一一,2,2,5.897154,0.002740,0.016157
一下,11,12,4.192406,0.016438,0.068916
一个,74,86,2.286236,0.117808,0.269337
一些,3,5,5.491689,0.006849,0.037614
一人,11,11,4.192406,0.015068,0.063173


In [19]:
tf_idf1 = tf_idf.reset_index()

In [20]:
tf_idf1.sort_values(by='score',ascending=False)

,0,1,2,idf,tf,score
2569,好好,246,292,1.084970,0.400000,0.433988
3589,我们,377,429,0.658056,0.587671,0.386720
5745,粮食,205,216,1.267291,0.295890,0.374979
1928,吃饭,271,277,0.988182,0.379452,0.374968
1334,先生,241,244,1.105504,0.334247,0.369511
...,...,...,...,...,...,...
2785,富饶,1,1,6.590301,0.001370,0.009028
4650,泼洒,1,1,6.590301,0.001370,0.009028
4651,泽之,1,1,6.590301,0.001370,0.009028
4091,是知,1,1,6.590301,0.001370,0.009028


In [21]:
tf_idf1[(tf_idf1[2]>=3) & (tf_idf1[1]>=3)].sort_values(by='score',ascending=False)

,0,1,2,idf,tf,score
2569,好好,246,292,1.084970,0.400000,0.433988
3589,我们,377,429,0.658056,0.587671,0.386720
5745,粮食,205,216,1.267291,0.295890,0.374979
1928,吃饭,271,277,0.988182,0.379452,0.374968
1334,先生,241,244,1.105504,0.334247,0.369511
...,...,...,...,...,...,...
4337,来接,3,3,5.491689,0.004110,0.022569
1021,会一,3,3,5.491689,0.004110,0.022569
2567,好大,3,3,5.491689,0.004110,0.022569
3629,我是,3,3,5.491689,0.004110,0.022569


In [22]:
tf_idf1.sort_values(by='score',ascending=False).to_csv('score.csv',index=False)

In [23]:
tf_idf1.sort_values(by=1,ascending=False).to_csv('idf.csv',index=False)